# Review

Main points:

    * First, what to look for?
    
        1. Articles that calculate (/report) the red marrow dose in patients
        2. Introduce or discuss a limit for RM dose
        

What have I done so far?

    1. Made some simple literature searches in pubmed and scopus
       "marrow AND dosimetry" yielded 1728 hits in scopus
       "marrow AND dosimetry AND SPECT" gave 35 hits
       "marrow AND dosimetry AND PET! gave 75 hits
       
    2. Made also a "wild goose chase" search that was unstructured,
       just jumped from article to article in a more tree-like fashion
    
    
For july, it is now the 11. I have to formalize and do some "real" work. What about using graph theory? Idea is to start with Bel

In [ ]:
# Using the scopus API with a pre-written python-wrapper to analyze further

from scopus.scopus_api import ScopusAbstract

ab = ScopusAbstract("2-s2.0-84961056233")

# It works! 

In [36]:
# Now see if we can just retreive from a list of articles:references

import pandas as pd
import numpy as np
import xlrd
import re

In [101]:
book = xlrd.open_workbook("benua_work_book.xlsx")

print book

sh = book.sheet_by_index(0)

print sh.ncols
print "This sheet has {0} references and {1} columns".format(sh.nrows, sh.ncols)

# Check a single reference

ref_num = 0

single_reference_row = sh.row(ref_num+1)

print len(single_reference_row)

# Extra care for the first "Mother-reference"
# First character before title is *alwas* a " " "
# First seven characters is a string descriptions

literal_string =  str(single_reference_row[0])

#print literal_string[0:8]
athour_index_start = 8
#print literal_string[athour_index_start::]
title_start_index = literal_string.index('"')
title_end_index = literal_string.index('"',title_start_index+3)

author_string = literal_string[athour_index_start-1:title_start_index]
title_string = literal_string[title_start_index+1:title_end_index] # Yields 

# Now make a list of authors...

at_list = author_string.split(',')
at_list = filter(None, at_list)

for i in range(len(at_list)):
    
    at_list[i] = at_list[i].replace(" ", "")
    
print at_list,title_string

# Mother reference is now successfully parsed(!) Now with the child-references

ref_num = 1
single_reference_row = sh.row(ref_num+1)

reference_string = str(single_reference_row[1])

#print at_list

#print reference_string
character_split = '.,'

sep_index = [m.start() for m in re.finditer('\.,', reference_string)]
year_index = [m.start() for m in re.finditer('\d\d\d\d', reference_string)]

#print reference_string[sep_index[-1]+3:year_index[0]-2] # Seems to work...

#print reference_string[sep_index[-1]::]

1074
This sheet has 243 references and 1074 columns
1074
['SarrutD.', 'BadelJ.-N.', 'HaltyA.', 'GarinG.', 'PerolD.', 'CassierP.', 'BlayJ.-Y.', 'KryzaD.', 'GiraudetA.-L.'] 3D absorbed dose distribution estimated by Monte Carlo simulation in radionuclide therapy with a monoclonal antibody targeting synovial sarcoma


In [118]:
def author_list_from_string(author_string):
    
    """Simple utility function that translates a string containing
       authors into a list of authors for better book-keeping"""
    
    at_list = author_string.split('.,')
    at_list = filter(None, at_list)

    for i in range(len(at_list)): # TODO: Could be iterated better
    
        at_elem = at_list[i]
        
        if at_elem[0] == ' ': # TODO: Could be made more elegant
            
            at_elem = at_elem[1::]
            
        if at_elem[-1] == ' ':
            
            at_elem = at_elem[0:-2]
            
        at_list[i] = at_elem
        
    return at_list
    
def reference_title_and_author(reference_string):
    
    """Function takes a reference imported from Scopus and parses a 
       list of authors and the title"""
    
    character_split = '.,'

    sep_index = [m.start() for m in re.finditer('\.,', reference_string)]
    year_index = [m.start() for m in re.finditer('\d\d\d\d', reference_string)]
    
    author_string = reference_string[8:sep_index[-1]]
    
    author_list = author_list_from_string(author_string)
    
    title = reference_string[sep_index[-1]+3:year_index[0]-2] # Seems to work...
    
    return title, author_list

def parse_mother_reference(reference_string):
    
    """Function to parse the "mother"-string. Meaning the first reference
       when the data is presented in a "article A cites article b, c,... 
       manner """
    
    title_start_index = literal_string.index('"')
    title_end_index = literal_string.index('"',title_start_index+3)

    author_string = literal_string[athour_index_start-1:title_start_index]
    title = literal_string[title_start_index+1:title_end_index]
    
    rest_of_reference = reference_string[title_end_index::]
    
    author_list = author_list_from_string(author_string)
    
    url_start_index = rest_of_reference.index("www.scopus.com")
    
    print rest_of_reference[url_start_index::]
    
    
    
    
    
    return title, author_list

ref_num = 0

single_reference_row = sh.row(ref_num+1)
reference_string = str(single_reference_row[0])

#print reference_string

print parse_mother_reference(reference_string)
    

www.scopus.com/inward/record.uri?eid=2-s2.0-85009999980&doi=10.1186%2fs40658-016-0172-1&partnerID=40&md5=3d9c1dd9012601ab0b7e82e039a47c72","Fukukawa, C., Hanaoka, H., Nagayama, S., Tsunoda, T., Toguchida, J., Endo, K., Nakamura, Y., Katagiri, T., Radioimmunotherapy of human synovial sarcoma using a monoclonal antibody against fzd10 (2008) Cancer Sci, 99 (2), pp. 432-440. , COI: 1:CAS:528:DC%2BD1MXjs1ygu7w%3D, PID: 18271942 10.1111/j.1349-7006.2007.00701.x'
('3D absorbed dose distribution estimated by Monte Carlo simulation in radionuclide therapy with a monoclonal antibody targeting synovial sarcoma', ['Sarrut D', 'Badel J.-N', 'Halty A', 'Garin G', 'Perol D', 'Cassier P', 'Blay J.-Y', 'Kryza D', 'Giraudet A.-L'])
